In [1]:
!nvidia-smi

Thu Jan 21 08:07:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# SETUP

In [2]:
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

In [3]:
!git clone https://github.com/nclibz/MRKnee/

Cloning into 'MRKnee'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 336 (delta 1), reused 1 (delta 0), pack-reused 328
Receiving objects: 100% (336/336), 126.39 MiB | 38.88 MiB/s, done.
Resolving deltas: 100% (217/217), done.


In [4]:
!pip install pytorch-lightning
!pip install timm
!pip install neptune-client
!pip install albumentations -U
!pip install neptune-contrib



     |████████████████████████████████| 686kB 9.3MB/s 
     |████████████████████████████████| 829kB 17.8MB/s 
     |████████████████████████████████| 645kB 37.7MB/s 
     |████████████████████████████████| 102kB 15.4MB/s 
     |████████████████████████████████| 1.3MB 51.3MB/s 
     |████████████████████████████████| 143kB 33.5MB/s 
     |████████████████████████████████| 296kB 51.3MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=a50db0cccff6aa8fd76d96b1c7219d2c4ec593a47afeb637a9fb67322a60a1b3
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 sha256=86b75dc4985556f008af8c13f159453d6a15675e9297f8c4b84d2e627734b78d
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built future idna-ssl
  Found existing installation: future 0.16.0
    Unin

# PATHS

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# setup wd and datadir
import os
os.chdir('/content/MRKnee/')
os.getcwd()



'/content/MRKnee'

## GIT

In [7]:
! git pull https://github.com/nclibz/MRKnee/
!git checkout v2

From https://github.com/nclibz/MRKnee
 * branch            HEAD       -> FETCH_HEAD
Already up to date.
Branch 'v2' set up to track remote branch 'v2' from 'origin'.
Switched to a new branch 'v2'


In [8]:
!git pull origin v2

From https://github.com/nclibz/MRKnee
 * branch            v2         -> FETCH_HEAD
Already up to date.


# MODEL

In [9]:
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from model import MRKnee
from data import MRKneeDataModule
import albumentations as A
from utils import print_top_losses


pl.seed_everything(123)

%load_ext autoreload
%autoreload 0




Global seed set to 123


In [ ]:

class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [ ]:

def objective(trial):

    IMG_SZ = 224  # b0 = 224, b1 = 240,

    cfg = {
        # DATA
        'datadir': 'data',
        'diagnosis': 'meniscus',
        'planes': ['axial'],  # , 'sagittal', 'coronal', 'axial',
        'n_chans': 1,
        'num_workers': 4,
        'pin_memory': True,
        'upsample': False,
        'w_loss': True,
        'indp_normalz': False,
        'transf': {
            'train': [A.Rotate(limit=25, p=1),
                      A.HorizontalFlip(p=0.5),
                      A.RandomCrop(IMG_SZ, IMG_SZ)],
            'valid': [A.CenterCrop(IMG_SZ, IMG_SZ)]
        },
        # MODEL
        'backbone': 'efficientnet_b0',
        'pretrained': True,
        'learning_rate': trial.suggest_loguniform('lr', 1e-6, 1e-2),
        'drop_rate': trial.suggest_float('dropout', 0., 0.8),
        'freeze_from': -1,
        'unfreeze_epoch': 0,
        'log_auc': True,
        'log_ind_loss': True,
        'final_pool': 'max',
        # Trainer
        'precision': 16,
        'max_epochs': 5,
    }

    # LOGGER
    neptune_logger = pl_loggers.NeptuneLogger(
        api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNDI5ODUwMzQtOTM0Mi00YTY2LWExYWQtMDNlZDZhY2NlYjUzIn0=",
        params=cfg,
        project_name='nclibz/optuna-test',
        tags=[cfg['diagnosis']] + cfg['planes']
    )

    # Callbacks
    model_checkpoint = ModelCheckpoint(dirpath=f'checkpoints/trial{trial.number}/',
                                       filename='{epoch:02d}-{val_loss:.2f}-{val_auc:.2f}',
                                       verbose=True,
                                       save_top_k=2,
                                       monitor='val_loss',
                                       mode='min',
                                       period=1)

    lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval="epoch")

    metrics_callback = MetricsCallback()

    prune_callback = PyTorchLightningPruningCallback(trial, monitor="val_loss")

    # DM AND MODEL
    dm = MRKneeDataModule(**cfg)
    model = MRKnee(**cfg)
    trainer = pl.Trainer(gpus=1,
                         precision=cfg['precision'],
                         max_epochs=cfg['max_epochs'],
                         logger=neptune_logger,
                         log_every_n_steps=100,
                         num_sanity_val_steps=0,
                         callbacks=[lr_monitor,
                                    model_checkpoint,
                                    metrics_callback,
                                    prune_callback],
                         progress_bar_refresh_rate=20,
                         limit_train_batches=0.10,  # HUSK AT SLETTE
                         deterministic=True)

    trainer.fit(model, dm)

    return metrics_callback.metrics[-1]["val_loss"].item()

https://ui.neptune.ai/nclibz/mrknee/e/MRKNEE-38


NeptuneLogger will work in online mode


In [ ]:


pruner = optuna.pruners.MedianPruner()
# skal vel også bruge en TPE sampler?
study = optuna.create_study(direction="minimize", pruner=pruner)

study.optimize(objective, n_trials=10, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
## UPLOAD BEST CHECKPOINTS TO LOG
neptune_logger.experiment.set_property('best_val_loss', model_checkpoint.best_model_score.tolist())
for k in model_checkpoint.best_k_models.keys():
    model_name = 'checkpoints/' + k.split('/')[-1]
    neptune_logger.experiment.log_artifact(k, model_name)




ChunkedEncodingError: ignored

In [ ]:
### UPLOAD  SAMPLE LOSSES  - tager dog den sidste og ik ved bedste val. Skal implementere inde i modellen
from neptunecontrib.api import log_pickle
log_pickle('v_sample_loss.pkl', model.v_sample_loss, neptune_logger)
log_pickle('t_sample_loss.pkl', model.t_sample_loss, neptune_logger)

In [ ]:
# save top losses to pkl files 
# uploade til neptune??
import pickle 

pickle.dump( model.t_sample_loss, open( "t_sample_loss.p", "wb" ) )
pickle.dump( model.v_sample_loss, open( "v_sample_loss.p", "wb" ) )
neptune_logger.log_artifact("t_sample_loss.p")
neptune_logger.log_artifact("v_sample_loss.p")


print_top_losses(model.t_sample_loss,5)

Sample : Loss
('0943',)  :  tensor(0.9017, device='cuda:0')
('0408',)  :  tensor(0.7901, device='cuda:0')
('1009',)  :  tensor(0.6418, device='cuda:0')
('0087',)  :  tensor(0.4506, device='cuda:0')
('0768',)  :  tensor(0.2992, device='cuda:0')


In [ ]:


neptune_logger.log_artifact(export_pickle(model.t_sample_loss), "t_sample_loss.pkl")
  



In [ ]:
model.trainer.callbacks.model_ch

In [ ]:
model.trainer.logger

TypeError: ignored

In [ ]:
model.best_val_loss

tensor(0.3541, device='cuda:0')